# Create MBTiles for USA Resilience Opportunity Map platform

Check this [notebook](https://github.com/Vizzuality/sci_team_data_bank/blob/master/Encyclopedia/map_tile_processing/MBTiles_from_Carto_data.ipynb) for further information on the creation of `MBTiles`.

## Table of Contents
### [Python libraries](#libraries)
### [Utils](#utils)
- **[create_mbtiles](#create_mbtiles)**

### [Create `MBTiles`](#create_mbtiles_2)
### [Show `MBTiles` in our localhost](#show_mbtiles)

<a id='libraries'></a>
### Python libraries

In [1]:
import pandas as pd 
import geopandas as gpd
import getpass
import requests
import shapely.wkb 
import subprocess

<a id='utils'></a>
### Utils

<a id='create_mbtiles'></a>
**create_mbtiles**

In [2]:
def create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel"):
    """
    Use tippecanoe to create a MBTILE at dest_path from source_path.
    layer_name is used for the name of the layer in the MBTILE.
    Regex file path (/*.geojson) is supported for source_path.
    """
    cmd = f"tippecanoe -o {dest_path} -l {layer_name} {opts} {source_path}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

<a id='read_data'></a>
### Read data from different sources 
**[County data](https://catalog.data.gov/dataset/tiger-line-shapefile-2016-nation-u-s-current-county-and-equivalent-national-shapefile)**

In [8]:
county = gpd.read_file('https://www2.census.gov/geo/tiger/TIGER2016/COUNTY/tl_2016_us_county.zip')
county = county.astype({'GEOID': 'int32'})
county = county[['GEOID', 'NAME', 'geometry']]
county.rename(columns={'GEOID': 'County FIPS code', 'NAME': 'Name'}, inplace= True)
county.head()

,County FIPS code,Name,geometry
0,31039,Cuming,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,53069,Wahkiakum,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."
2,35011,De Baca,"POLYGON ((-104.56739 33.99757, -104.56772 33.9..."
3,31109,Lancaster,"POLYGON ((-96.91060 40.95841, -96.91060 40.958..."
4,31129,Nuckolls,"POLYGON ((-98.27367 40.08940, -98.27367 40.089..."


In [6]:
df = pd.read_csv('http://www.globalpolicy.science/s/county_damages_by_sector.csv')

In [9]:
df = pd.merge(left=df, right=county, how='left', left_on='County FIPS code', right_on='County FIPS code')
df = gpd.GeoDataFrame(df)
df.head()

,State Code,County Name,County FIPS code,County Population (in 2012),County Income (in 2012),Agricultural Damage (%) (4 major crops),Mortality (deaths per 100k),Energy Expenditures (%),Labor Low-risk (%),Labor High-risk (%),Coastal damage (log10(% county income)),Property Crime (%),Violent Crime (%),Total damages (% county income),Name,geometry
0,AL,Autauga County,1001,55192,35106,-16.003536,27.024584,11.998686,-0.515579,-2.120601,-2.051729,0.052951,2.369668,9.4390,Autauga,"POLYGON ((-86.90310 32.54063, -86.90313 32.541..."
1,AL,Baldwin County,1003,190560,38228,-12.974306,31.895386,12.066700,-0.378806,-1.937923,-0.698060,0.147257,2.495164,9.4078,Baldwin,"POLYGON ((-87.99068 30.55549, -87.99051 30.560..."
2,AL,Barbour County,1005,27169,28431,-6.562765,25.534552,11.380579,-0.394443,-1.976559,-1.961138,0.151926,2.528313,10.8720,Barbour,"POLYGON ((-85.42982 32.04598, -85.42985 32.046..."
3,AL,Bibb County,1007,22634,27109,NaN,22.530268,12.070424,-0.428851,-1.934934,-1.940463,0.187312,2.525677,10.7770,Bibb,"POLYGON ((-87.31226 33.08622, -87.31218 33.087..."
4,AL,Blount County,1009,57748,29513,-18.488190,20.003851,11.908128,-0.374522,-1.854808,-1.976278,0.283651,2.659189,8.9849,Blount,"POLYGON ((-86.74919 33.99760, -86.74902 33.997..."


In [12]:
df.to_file("../data/county_damages_by_sector.json", driver="GeoJSON")

<a id='create_mbtiles_2'></a>
### Create `MBTiles`

In [3]:
layer_name = "Indicators"
source_path = "../data/indicators.json"
dest_path = "../data/indicators.mbtiles"
create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")

Processing: tippecanoe -o ../data/indicators.mbtiles -l Indicators -zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel ../data/indicators.json
Task created
Finished processing


<a id='show_mbtiles'></a>
### Show `MBTiles` in our localhost

In [4]:
!mbview --port 9000 ../data/indicators.mbtiles

Listening on http://localhost:9000
^C
